# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [216]:
from analysis import MatchAnalysis
import pandas as pd
import util_strings as utils
from ranking import Ranking
from football_predictions import FootballPredictions

In [217]:
championship = pd.read_csv(utils.championship_actual_season, index_col=0)
championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

In [218]:
matches = new_matches[:10][['team1', 'team2', 'date', 'result']]
matches.rename(columns= {'team1': 'home', 'team2': 'away'}, inplace=True)
matches.reset_index(drop=True, inplace=True)


## Ottenimento testo FootballPrediction
Scaricamento del testo di ciascuna partita

In [220]:
fp = FootballPredictions(matches=matches)
fp.update_originalNotation()
fp.get_urls()
fp.get_predictions(utils.next_matches)
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates(utils.next_matches)

100%|██████████| 4/4 [00:12<00:00,  3.01s/it]


In [221]:
from my_tokenizer import MyTokenizer

mt = MyTokenizer(fp.df)
mt.clean_text()
mt.open_vectorizer(utils.vectorizer)

In [222]:
import pickle

dt = pickle.load(open('ML model/ta decision tree 0.942 (no syn).model', 'rb'))
predictions = dt.predict(mt.vectorizer.transform(mt.cleaned_corpus))

In [223]:
cont = 0
for i, el in fp.df.iterrows():
    if (el.prediction == predictions[i]):
        cont += 1
          
print(cont)

7


Aggiungo le predizioni al dataset dei match

In [224]:
cont = 0
for i, match in matches.iterrows():
    matches.at[i, 'prediction'] = predictions[cont]
    cont += 1

In [225]:
matches

,home,away,date,result,prediction
14,Juventus,Empoli,2022-10-21,nan,N
11,Salernitana,Spezia,2022-10-22,nan,V
16,Fiorentina,Inter,2022-10-22,nan,N
14,Milan,Monza,2022-10-22,nan,N
10,Atalanta,Lazio,2022-10-23,nan,N
12,Bologna,Lecce,2022-10-23,nan,V
14,Roma,Napoli,2022-10-23,nan,N
12,Udinese,Torino,2022-10-23,nan,V
11,Sassuolo,Hellas Verona,2022-10-24,nan,V
12,Cremonese,Sampdoria,2022-10-24,nan,V


Per ciascun match devo calcolare le statistiche date dalle medie dei 5 match precedenti e poi devo ottenere il testo da football_prediction

In [ ]:
ranking = Ranking('SerieA', '2022-2023')
ranking.read_matches(seasons=4, path=utils.ranking)

ma = MatchAnalysis()
ma.set_ranking(ranking=ranking)
#leggo tutti i match (doppi perché ci sono le statistiche x ogni squadra)
ma.read_matches(utils.merged_statistics)
#creo una lista di oggetti (lista di squadre)
ma.create_team_dataset() 

matches['date'] = pd.to_datetime(matches['date'], format='%Y-%m-%d') 
float_features_and_avg = ['goals','total_shots','shots_on_target','goals_on_penalty','total_penalties','percentage_possession','fouls','yellow_cards','red_cards']

import numpy as np
for f in float_features_and_avg:
    matches[f] = np.nan

matches[float_features_and_avg] = matches[float_features_and_avg].astype(float)

for i, match in matches.iterrows(): 
    avgH, c = ma.get_team_by_name(match.home).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.away).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    
    for col in float_features_and_avg:
        diff = avgH[col] - avgA[col]
        match[col] = diff
        matches.at[i, col] = diff

matches

,home,away,date,result,prediction,goals,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards
14,Juventus,Empoli,2022-10-21,nan,N,-1.2,-7.0,-3.0,0.0,0.2,-5.2,6.4,1.0,0.0
11,Salernitana,Spezia,2022-10-22,nan,V,1.2,1.2,2.6,0.2,0.2,-2.2,-8.0,-1.6,0.0
16,Fiorentina,Inter,2022-10-22,nan,N,-0.6,0.0,1.0,0.0,0.2,1.6,0.6,1.4,0.2
14,Milan,Monza,2022-10-22,nan,N,-1.2,-7.0,-3.0,0.0,0.2,-5.2,6.4,1.0,0.0
10,Atalanta,Lazio,2022-10-23,nan,N,-1.4,-0.6,-1.2,0.0,-0.2,-7.6,5.2,2.2,0.0
12,Bologna,Lecce,2022-10-23,nan,V,0.4,6.6,-0.2,0.2,0.2,-11.0,-2.6,-2.0,0.0
14,Roma,Napoli,2022-10-23,nan,N,-1.2,-7.0,-3.0,0.0,0.2,-5.2,6.4,1.0,0.0
12,Udinese,Torino,2022-10-23,nan,V,0.4,6.6,-0.2,0.2,0.2,-11.0,-2.6,-2.0,0.0
11,Sassuolo,Hellas Verona,2022-10-24,nan,V,1.2,1.2,2.6,0.2,0.2,-2.2,-8.0,-1.6,0.0
12,Cremonese,Sampdoria,2022-10-24,nan,V,0.4,6.6,-0.2,0.2,0.2,-11.0,-2.6,-2.0,0.0


In [ ]:
#r = Ranking("Serie A", '2022-2023')
#r.read_matches(seasons=5)
ma = MatchAnalysis("files/Serie A/3 seasons.csv")
#ma.set_ranking(r)
ma.read_diff_dataset()

ma.matches_fe_team['date'] = pd.to_datetime(ma.matches_fe_team['date'], format='%Y-%m-%d')
ma.create_team_dataset()

float_features_and_avg = [x for x in ma.matches_by_team[0].matches.columns if x != 'team1' and x != 'team2' and x != 'date' and x != 'result' and x != 'rank_h' and x != 'rank_a' and x != 'matchday' and x != 'stadium' and x != 'completed_passings' and x != 'total_passings' and x != 'corners']

for i, match in matches.iterrows(): 
    avgH, c = ma.get_team_by_name(match.team1).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.team2).get_avg_last_X_matches(5, match.date, float_features_and_avg)

    for col in float_features_and_avg: 
        diff = avgH[col] - avgA[col]
        matches.at[i, col] = diff

matches

TypeError: __init__() takes 1 positional argument but 2 were given